# HlyB All Trajectory Analysis

## MD Analysis

### Import Modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statistics as s
import MDAnalysis as mda
import random

### Helper Modules

In [ ]:
# Converts 3 letter code to 1 letter code
convert_to_one_letter_dic = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G",\
                            "HSD":"H", "HIS": "H", "HSE":"H", "ILE": "I", "LEU": "L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P",\
                            "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V"}
def convert_to_one_letter(string):
    amino_acid = str(string[0:3])
    return convert_to_one_letter_dic[amino_acid]

In [ ]:
# Permutation test function
def permutation(pooled,sizeA,sizeB,iterations):
    """
    Function takes pooled samples and randomly shuffles them according to partition size.
    New randommly shuffled partitions are sorted and substracted mean calculated (delta).
    Delta appended to a list. 
    Set x iterations.
    """
    count = 0
    mean_list = []
    while count < iterations:
        np.random.shuffle(pooled)
        new_A = pooled[:sizeA]
        new_B = pooled[-sizeB:]
        delta = s.mean(new_A) - s.mean(new_B)
        mean_list.append(delta)
        count += 1
    return mean_list

In [ ]:
def extract_chain_vals_temp(data):
    """
    Function takes a dataframe and returns substrate or no substrate Lipid_Count values separated by temperature.
    """
    sub_vals_30_deg = []
    sub_vals_37_deg = []
    no_sub_vals_30_deg = []
    no_sub_vals_37_deg = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            if row._3 == 303.15:
                sub_vals_30_deg.append(row.Lipid_Count)
            elif row._3 == 310.15:
                sub_vals_37_deg.append(row.Lipid_Count)
        elif row._2 == 'No':
            if row._3 == 303.15:
                no_sub_vals_30_deg.append(row.Lipid_Count)
            elif row._3 == 310.15:
                no_sub_vals_37_deg.append(row.Lipid_Count)
                
    return sub_vals_30_deg, sub_vals_37_deg, no_sub_vals_30_deg, no_sub_vals_37_deg

In [ ]:
def extract_chain_vals(data):
    """
    Function takes a dataframe and returns substrate or no substrate Lipid_Count values.
    """
    sub_vals = []
    no_sub_vals = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            sub_vals.append(row.Lipid_Count)
        elif row._2 == 'No':
            no_sub_vals.append(row.Lipid_Count)
                
    return sub_vals, no_sub_vals

In [ ]:
def chain_dic_frames(data):
    """
    Function takes a dataframe and returns substrate or no substrate lipid interaction values based on chainID.
    """
    sub_dic = dict()
    nosub_dic = dict()
    for row in data.itertuples():
        if row._1 == "Yes":
            sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_Lipid_Count
        elif row._1 == "No":
            nosub_dic[(row.Chain,row.Joint_Position)] = row.Mean_Lipid_Count
                
    return sub_dic, nosub_dic

In [ ]:
def chain_pair_vals(chain_A,chain_B):
    """
    Function takes dictionary of chain values for chain A and B and returns the difference
    when the amino acids match. Else searches for instances where amino acid is unique to chain A or B.
    """
    A_B_diff = dict()
    unique_A = dict()
    unique_B = dict()
    for k1,v1 in chain_A.items():
        for k2,v2 in chain_B.items():
            if k1 == k2:
                diff = 0
                diff = v1 - v2
                A_B_diff[k1] = diff
            elif k1 not in chain_B.keys():
                unique_A[k1] = v1
    for k3,v3 in chain_B.items():
        if k3 not in chain_A.keys():
            unique_B[k3] = v3
    return A_B_diff, unique_A, unique_B

In [ ]:
def bootstrap_lipid(pooled,n_samples,iterations):
    random.seed(42) # set seed
    count = 0
    mean_diff_list = [] #store median difference values for plotting
    while count < iterations:
        # randomly select samples from the list without replacement
        # calculate mean difference - smoother distn than calc median diff
        np.random.shuffle(pooled) # first shuffle the data
        sample_1 = random.sample(pooled, n_samples)
        sample_2 = random.sample(pooled, n_samples)
        mean_diff = np.mean(sample_1) - np.mean(sample_2)
        mean_diff_list.append(mean_diff)
        count += 1
        
    return mean_diff_list

In [ ]:
def extract_lipid_vals_cdl2(data):
    """
    Function takes a dataframe and returns values for CDL2 for either substrate or no substrate.
    """
    cdl2_vals_sub = []
    cdl2_vals_no_sub = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            cdl2_vals_sub.append(row.CDL2)
        elif row._2 == 'No':
            cdl2_vals_no_sub.append(row.CDL2)
    return cdl2_vals_sub, cdl2_vals_no_sub

In [ ]:
def extract_lipid_vals_temp_cdl2(data):
    """
    Function takes a dataframe and returns values for CDL2 for either substrate or no substrate based on temp.
    """
    cdl2_vals_sub_30_deg = []
    cdl2_vals_sub_37_deg = []
    cdl2_vals_no_sub_30_deg = []
    cdl2_vals_no_sub_37_deg = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            if row._3 == 303.15:
                cdl2_vals_sub_30_deg.append(row.CDL2)
            elif row._3 == 310.15:
                cdl2_vals_sub_37_deg.append(row.CDL2)
        elif row._2 == 'No':
            if row._3 == 303.15:
                cdl2_vals_no_sub_30_deg.append(row.CDL2)
            elif row._3 == 310.15:
                cdl2_vals_no_sub_37_deg.append(row.CDL2)
                
    return cdl2_vals_sub_30_deg, cdl2_vals_sub_37_deg, cdl2_vals_no_sub_30_deg, cdl2_vals_no_sub_37_deg

In [ ]:
def p_val(difference_dic,iterations,permute_results):
    results_array = np.array(permute_results)
    p_val_dict = dict()
    for v1, v2 in difference_dic.items():
        if v2 < 0:
            diffCount = len(np.where(np.sort(results_array) <= v2)[0])
            p_val = float(diffCount)/float(iterations)
            p_val_dict[v1] = p_val
        else:
            diffCount = len(np.where(np.sort(results_array) >= v2)[0])
            p_val = float(diffCount)/float(iterations)
            p_val_dict[v1] = p_val
    return p_val_dict

In [ ]:
def sig_p_val(p_val_dic,sig_level):
    num_tests = int(len(p_val_dic))
    bonferroni = sig_level/num_tests
    sig_vals = dict()
    for key,val in p_val_dic.items():
        if val <= bonferroni:
            sig_vals[key] = val
    return sig_vals

In [ ]:
def lipid_dic_cdl2(data):
    """
    Function takes a dataframe and returns cdl2 interaction values as a dictionary
    by amino acid position and chain ID
    """
    cdl2_sub_dic = dict()
    cdl2_no_sub_dic = dict()
    for row in data.itertuples():
        if row._1 == 'Yes':
            cdl2_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_CDL2_Count
        elif row._1 == 'No':
            cdl2_no_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_CDL2_Count
            
    return cdl2_sub_dic, cdl2_no_sub_dic

In [ ]:
def extract_lipid_vals_pope_popg(data):
    """
    Function takes a dataframe and returns values for POPE/POPG for either substrate or no substrate.
    """
    pope_popg_vals_sub = []
    pope_popg_vals_no_sub = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            pope_popg_vals_sub.append(row.POPE + row.POPG)
        elif row._2 == 'No':
            pope_popg_vals_no_sub.append(row.POPE + row.POPG)
    return pope_popg_vals_sub, pope_popg_vals_no_sub

In [ ]:
def extract_lipid_vals_temp_pope_popg(data):
    """
    Function takes a dataframe and returns values for POPE/POPG for either substrate or no substrate based on temp.
    """
    pope_popg_vals_sub_30_deg = []
    pope_popg_vals_sub_37_deg = []
    pope_popg_vals_no_sub_30_deg = []
    pope_popg_vals_no_sub_37_deg = []
    for row in data.itertuples():
        if row._2 == 'Yes':
            if row._3 == 303.15:
                pope_popg_vals_sub_30_deg.append(row.POPE + row.POPG)
            elif row._3 == 310.15:
                pope_popg_vals_sub_37_deg.append(row.POPE + row.POPG)
        elif row._2 == 'No':
            if row._3 == 303.15:
                pope_popg_vals_no_sub_30_deg.append(row.POPE + row.POPG)
            elif row._3 == 310.15:
                pope_popg_vals_no_sub_37_deg.append(row.POPE + row.POPG)
                
    return pope_popg_vals_sub_30_deg, pope_popg_vals_sub_37_deg, pope_popg_vals_no_sub_30_deg, pope_popg_vals_no_sub_37_deg

In [ ]:
def lipid_dic_pope_popg(data):
    """
    Function takes a dataframe and returns cdl2 interaction values as a dictionary
    by amino acid position and chain ID
    """
    pope_popg_sub_dic = dict()
    pope_popg_no_sub_dic = dict()
    for row in data.itertuples():
        if row._1 == 'Yes':
            pope_popg_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_POPE_POPG_Count
        elif row._1 == 'No':
            pope_popg_no_sub_dic[(row.Chain,row.Joint_Position)] = row.Mean_POPE_POPG_Count
            
    return pope_popg_sub_dic, pope_popg_no_sub_dic

In [ ]:
def extract_sub_vals_clust(data):
    """
    Function takes a dataframe and returns substrate or no substrate average median count values.
    """
    sub_vals = []
    no_sub_vals = []
    for row in data.itertuples():
        if row.Substrate == 'yes':
            sub_vals.append(row.Scaled_Median_Count)
        elif row.Substrate  == 'no':
            no_sub_vals.append(row.Scaled_Median_Count)
                
    return sub_vals, no_sub_vals

### Plots for thesis - set colours

In [ ]:
palette = sns.color_palette("deep")

In [ ]:
blue = sns.color_palette("deep")[0]
orange = sns.color_palette("deep")[1]
green = sns.color_palette("deep")[2]
red = sns.color_palette("deep")[3]
purple = sns.color_palette("deep")[4]
brown = sns.color_palette("deep")[5]
pink = sns.color_palette("deep")[6]
grey = sns.color_palette("deep")[7]
gold = sns.color_palette("deep")[8]
turqoise = sns.color_palette("deep")[9]

### Define Universes

In [ ]:
# Read trajectory - ignore warning as this is coarse grain
u1 = mda.Universe("HlyB_closed_sub_30_deg_traj_1_chainAB.pdb","HlyB_closed_sub_30_deg_skip1000_chainAB_traj_1.xtc")
u2 = mda.Universe("HlyB_closed_sub_30_deg_traj_2_chainAB.pdb","HlyB_closed_sub_30_deg_skip1000_chainAB_traj_2.xtc")
u3 = mda.Universe("HlyB_closed_sub_37_deg_traj_1_chainAB.pdb","HlyB_closed_sub_37_deg_skip1000_chainAB_traj_1.xtc")
u4 = mda.Universe("HlyB_closed_sub_37_deg_traj_2_chainAB.pdb","HlyB_closed_sub_37_deg_skip1000_chainAB_traj_2.xtc")
u5 = mda.Universe("HlyB_closed_nosub_30_deg_traj_1_chainAB.pdb","HlyB_closed_nosub_30_deg_skip1000_chainAB_traj_1.xtc")
u6 = mda.Universe("HlyB_closed_nosub_30_deg_traj_2_chainAB.pdb","HlyB_closed_nosub_30_deg_skip1000_chainAB_traj_2.xtc")
u7 = mda.Universe("HlyB_closed_nosub_37_deg_traj_1_chainAB.pdb","HlyB_closed_nosub_37_deg_skip1000_chainAB_traj_1.xtc")
u8 = mda.Universe("HlyB_closed_nosub_37_deg_traj_2_chainAB.pdb","HlyB_closed_nosub_37_deg_skip1000_chainAB_traj_2.xtc")

### Ensemble Cluster Analysis

This analysis only works with preselected atoms in trajectories. Only reduce number of frames to speed up calculation.

In [ ]:
# import modules
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm

In [ ]:
# Cluster
ces0, details0 = encore.ces(ensembles=[u1,u2,u3,u4,u5,u6,u7,u8], select="type B")

In [ ]:
# Print cluster info
cluster_collection = details0['clustering'][0]
print(type(cluster_collection))
print('We have found {} clusters'.format(len(cluster_collection)))

In [ ]:
# k-means clustering
km1 = clm.KMeans(128,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km2 = clm.KMeans(64,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km3 = clm.KMeans(32,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default

km4 = clm.KMeans(16,  # no. clusters
                 init = 'k-means++',  # default
                 algorithm="auto")    # default



In [ ]:
ces2, details2 = encore.ces([u1,u2,u3,u4,u5, u6, u7, u8],
                         select='type B',
                         clustering_method=[km1, km2, km3, km4],
                         ncores=4)
print(len(ces2), len(details2['clustering']))

In [ ]:
# Plot clustering results
titles = ['Kmeans 128 clusters', 'Kmeans 64 clusters', 'Kmeans 32 clusters', 'Kmeans 16 clusters']
fig2, axes = plt.subplots(1, 4, sharey=True, figsize=(15, 4))
for i, (data, title) in enumerate(zip(ces2, titles)):
    imi = axes[i].imshow(data, vmax=np.log(2), vmin=0)
    axes[i].set_xticks(np.arange(8))
    axes[i].set_xticklabels(labels)
    axes[i].set_title(title)
plt.yticks(np.arange(8), labels)
cbar2 = fig2.colorbar(imi, ax=axes.ravel().tolist())
cbar2.set_label('Jensen-Shannon divergence')
plt.savefig("Ensemble Cluster K means All Trajectories.png", dpi = 300)

In [ ]:
#estimate errors
avgs, stds = encore.ces([u1, u2, u3, u4, u5, u6, u7, u8],
                         select='type B',
                         clustering_method=km3,
                         estimate_error=True,
                         ncores=4)

In [ ]:
avgs

In [ ]:
stds

## Protein Lipid Contacts

##### Raw Data 

In [ ]:
# Read in csv's
df_1 = pd.read_csv("data/HlyB_sub_30_deg_traj_1_protein_lipid_contact.csv")
df_2 = pd.read_csv("data/HlyB_sub_30_deg_traj_2_protein_lipid_contact.csv")
df_3 = pd.read_csv("data/HlyB_sub_37_deg_traj_1_protein_lipid_contact.csv")
df_4 = pd.read_csv("data/HlyB_sub_37_deg_traj_2_protein_lipid_contact.csv")
df_5 = pd.read_csv("data/HlyB_nosub_30_deg_traj_1_protein_lipid_contact.csv")
df_6 = pd.read_csv("data/HlyB_nosub_30_deg_traj_2_protein_lipid_contact.csv")
df_7 = pd.read_csv("data/HlyB_nosub_37_deg_traj_1_protein_lipid_contact.csv")
df_8 = pd.read_csv("data/HlyB_nosub_37_deg_traj_2_protein_lipid_contact.csv")

#### Combine Data

In [ ]:
frames = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8]
combined_df = pd.concat(frames, ignore_index=True)

In [ ]:
combined_df

In [ ]:
#Add one letter code back in
combined_df["Protein_Code"] = combined_df["Protein_Type"].apply(convert_to_one_letter)

In [ ]:
combined_df["Joint_Position"] = combined_df["Protein_Code"].astype(str) + combined_df["Protein_ResID"].astype(str)

In [ ]:
# First group by ID to sum appropriate lipid values
group_contact = combined_df.groupby(["ID","Substrate?","Temperature(K)","Chain","Protein_ResID","Joint_Position", "Lipid_Type"])["Fraction_Frames"].sum().unstack()

In [ ]:
group_contact.reset_index(inplace = True) #Reset index to remove multi-indexing

In [ ]:
#Fill in NaNs with 0s
group_contact["CDL2"].fillna(value = 0, inplace = True)
group_contact["POPE"].fillna(value = 0, inplace = True)
group_contact["POPG"].fillna(value = 0, inplace = True)

In [ ]:
#sort values by protein position
group_contact.sort_values("Protein_ResID", axis = 0, ascending = True, inplace = True) 

In [ ]:
# Add in sub columns for plotting
group_contact["Lipid_Count"] = group_contact["CDL2"] + group_contact["POPE"] + group_contact["POPG"]
group_contact["POPE_POPG"] = group_contact["POPE"] + group_contact["POPG"]
group_contact["Substrate?_Temperature"] = group_contact["Substrate?"].astype(str) + "_" + group_contact["Temperature(K)"].astype(str)
group_contact["Substrate?_Temperature_Chain"] = group_contact["Substrate?"].astype(str) + "_" + group_contact["Temperature(K)"].astype(str) + "_" + group_contact["Chain"].astype(str)
group_contact["Substrate?_Chain"] = group_contact["Substrate?"].astype(str) + "_" + group_contact["Chain"].astype(str)

In [ ]:
# Aggregate data - only separate by presence of substrate
group_contact_cdl2_lipid = group_contact.groupby(["ID","Temperature(K)","Substrate?","Chain","Protein_ResID","Joint_Position"])["CDL2"].mean().unstack()
group_contact_pope_popg_lipid = group_contact.groupby(["ID","Temperature(K)","Substrate?","Chain","Protein_ResID","Joint_Position"])["POPE_POPG"].mean().unstack()

In [ ]:
#Reset index to remove multi-indexing
group_contact_cdl2_lipid.reset_index(inplace = True)
group_contact_pope_popg_lipid.reset_index(inplace = True)

In [ ]:
# remove extra headers - cardiolipin
head_list_cdl2_lipid = group_contact_cdl2_lipid.columns.tolist()
head_list_cdl2_lipid.remove('ID')
head_list_cdl2_lipid.remove('Temperature(K)')
head_list_cdl2_lipid.remove('Substrate?')
head_list_cdl2_lipid.remove('Chain')
head_list_cdl2_lipid.remove('Protein_ResID')

In [ ]:
# remove extra headers pope/popg
head_list_pope_popg_lipid = group_contact_pope_popg_lipid.columns.tolist()
head_list_pope_popg_lipid.remove('ID')
head_list_pope_popg_lipid.remove('Temperature(K)')
head_list_pope_popg_lipid.remove('Substrate?')
head_list_pope_popg_lipid.remove('Chain')
head_list_pope_popg_lipid.remove('Protein_ResID')

In [ ]:
# Apply melt
melt_group_contact_cdl2 = group_contact_cdl2_lipid.melt(id_vars=['ID','Temperature(K)','Substrate?','Chain','Protein_ResID'],value_vars=head_list_cdl2_lipid,value_name="Mean_CDL2_Count")
melt_group_contact_pope_popg = group_contact_pope_popg_lipid.melt(id_vars=['ID','Temperature(K)','Substrate?','Chain','Protein_ResID'],value_vars=head_list_pope_popg_lipid,value_name="Mean_POPE_POPG_Count")

In [ ]:
# Remove nan values
melt_group_contact_cdl2.dropna(inplace = True)
melt_group_contact_pope_popg.dropna(inplace = True)

In [ ]:
# Sort for plotting - by protein residue number
melt_group_contact_cdl2.sort_values(["Protein_ResID","Chain"], axis = 0, ascending = True, inplace = True) 
melt_group_contact_pope_popg.sort_values(["Protein_ResID","Chain"], axis = 0, ascending = True, inplace = True)

In [ ]:
# Separate by substrate and chain ID
melt_group_contact_cdl2["Substrate?_Chain"] = melt_group_contact_cdl2["Substrate?"].astype(str) + "_" +  melt_group_contact_cdl2["Chain"].astype(str)
melt_group_contact_pope_popg["Substrate?_Chain"] = melt_group_contact_pope_popg["Substrate?"].astype(str) + "_" +  melt_group_contact_pope_popg["Chain"].astype(str)

In [ ]:
# Remove small values
melt_group_contact_cdl2_remove = melt_group_contact_cdl2[melt_group_contact_cdl2["Mean_CDL2_Count"]>0.01]
melt_group_contact_pope_popg_remove = melt_group_contact_pope_popg[melt_group_contact_pope_popg["Mean_POPE_POPG_Count"]>0.01]

Plotting when small values are removed will result in an incorrect plot. We will first define the positions which contain significantly large values, and remove others from the original "melt_group_contact_{lipid}" dataframe.

In [ ]:
# Define dictionary to store positions with significant cdl2 contacts
x_axis_headers_cdl2 = []
for row in melt_group_contact_cdl2_remove.itertuples():
    if row.Joint_Position not in x_axis_headers_cdl2:
        x_axis_headers_cdl2.append(row.Joint_Position)

In [ ]:
# Define dictionary to store positions with significant pope/popg contacts
x_axis_headers_popeg = []
for row in melt_group_contact_pope_popg_remove.itertuples():
    if row.Joint_Position not in x_axis_headers_popeg:
        x_axis_headers_popeg.append(row.Joint_Position)

In [ ]:
# Use these to make a new dataframe - cdl2
cdl2_ID_list = []
cdl2_temp_list = []
cdl2_sub_list = []
cdl2_chain_list = []
cdl2_resid_list = []
cdl2_joint_position_list = []
cdl2_list = []
cdl2_label_list = []

In [ ]:
# Use these to make a new dataframe - pope/popg
pope_popg_ID_list = []
pope_popg_temp_list = []
pope_popg_sub_list = []
pope_popg_chain_list = []
pope_popg_resid_list = []
pope_popg_joint_position_list = []
popeg_list = []
pope_popg_label_list = []

In [ ]:
# Loop over original dataframe and extract data from positions with significant cdl2 contacts
for row in melt_group_contact_cdl2.itertuples():
    if row.Joint_Position in x_axis_headers_cdl2:
        cdl2_ID_list.append(row.ID)
        cdl2_temp_list.append(row._2)
        cdl2_sub_list.append(row._3)
        cdl2_chain_list.append(row.Chain)
        cdl2_resid_list.append(row.Protein_ResID)
        cdl2_joint_position_list.append(row.Joint_Position)
        cdl2_list.append(row.Mean_CDL2_Count)
        cdl2_label_list.append(row._8)

In [ ]:
# Loop over original dataframe and extract data from positions with significant pope/popg contacts
for row in melt_group_contact_pope_popg.itertuples():
    if row.Joint_Position in x_axis_headers_popeg:
        pope_popg_ID_list.append(row.ID)
        pope_popg_temp_list.append(row._2)
        pope_popg_sub_list.append(row._3)
        pope_popg_chain_list.append(row.Chain)
        pope_popg_resid_list.append(row.Protein_ResID)
        pope_popg_joint_position_list.append(row.Joint_Position)
        popeg_list.append(row.Mean_POPE_POPG_Count)
        pope_popg_label_list.append(row._8)

In [ ]:
# Make a dictionary of the lists with column headings - cdl2
data_cdl2 = {'ID':cdl2_ID_list, 'Temperature(K)':cdl2_temp_list, 'Substrate?':cdl2_sub_list,'Chain':cdl2_chain_list,
        'Protein_ResID':cdl2_resid_list,'Joint_Position':cdl2_joint_position_list,
        'Mean_CDL2_Count':cdl2_list, 'Substrate?_Chain':cdl2_label_list}

In [ ]:
# Make a dictionary of the lists with column headings - pope/popg
data_pope_popg = {'ID':pope_popg_ID_list, 'Temperature(K)':pope_popg_temp_list, 'Substrate?':pope_popg_sub_list,
                  'Chain':pope_popg_chain_list, 'Protein_ResID':pope_popg_resid_list,
                  'Joint_Position':pope_popg_joint_position_list, 'Mean_POPE_POPG_Count':popeg_list, 
                  'Substrate?_Chain':pope_popg_label_list}

In [ ]:
# Use dictionaries to make new dataframes
cdl2_new = pd.DataFrame.from_dict(data_cdl2)
popeg_new = pd.DataFrame.from_dict(data_pope_popg)

##### Plots for thesis

In [ ]:
palette_dic = {"Yes_A":blue, "No_A":orange,"Yes_B":green,"No_B":red}

In [ ]:
# Plot using seaborn - cdl2
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
melt_group_contact_cdl2_fig = sns.barplot(x = "Joint_Position", y = "Mean_CDL2_Count", 
                                          hue = "Substrate?_Chain", hue_order=["Yes_A","No_A","Yes_B","No_B"], 
                                          data = cdl2_new, palette=palette_dic, estimator=np.mean,ci=90, 
                                          ax=ax).get_figure()
plt.xlabel("ResID")
plt.ylabel("CDL2 Fraction Frames")
plt.legend(loc='upper left');
melt_group_contact_cdl2_fig.savefig("Mean CDL2 Count by Substrate and Chain Relabel Remove.png", dpi = 300)

In [ ]:
# Plot using seaborn - pope/popg
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
melt_group_contact_popeg_fig = sns.barplot(x = "Joint_Position", y = "Mean_POPE_POPG_Count", 
                                           hue = "Substrate?_Chain", hue_order=["Yes_A","No_A","Yes_B","No_B"], 
                                           data = popeg_new, palette=palette_dic, estimator=np.mean,ci=90, 
                                           ax=ax).get_figure()
plt.xlabel("ResID")
plt.ylabel("POPE+POPG Fraction Frames")
plt.legend(loc='upper left');
melt_group_contact_popeg_fig.savefig("Mean POPE POPG Count by Substrate and Chain Relabel Remove.png", dpi = 300)

##### Exploratory Plots

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_lip_fig = sns.barplot(x = "Joint_Position", y = "Lipid_Count", hue = "Substrate?_Chain", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_lip_fig.savefig("Lipid Count Substrate and Chain.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_lip_fig = sns.barplot(x = "Joint Position", y = "Lipid Count", hue = "Substrate?_Temperature", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_lip_fig.savefig("Lipid Count Substrate and Temp.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_sub_fig = sns.barplot(x = "Joint Position", y = "Lipid Count", hue = "Substrate?", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_sub_fig.savefig("Lipid Count Substrate.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_temp_fig = sns.barplot(x = "Joint Position", y = "Lipid Count", hue = "Temperature (K)", data = group_contact_lipid_count, ax=ax).get_figure();
group_contact_temp_fig.savefig("Lipid Count Temperature.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_cdl2_fig = sns.barplot(x = "Joint_Position", y = "CDL2", hue = "Substrate?_Chain", data = group_contact_cdl2, palette=palette_dic, estimator=np.mean,ci=90, ax=ax).get_figure()
plt.xlabel("ResID")
plt.ylabel("CDL2 Fraction Frames")
plt.legend(loc='upper left');
group_contact_cdl2_fig.savefig("CDL2 Count Substrate and Chain Remove Small Values Relabel Estimate.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_pope_fig = sns.barplot(x = "Joint Position", y = "POPE", hue = "Temperature (K)", data = group_contact_pope, ax=ax).get_figure();
group_contact_pope_fig.savefig("POPE Count Temperature.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_popg_fig = sns.barplot(x = "Joint Position", y = "POPG", hue = "Substrate?_Temperature", data = group_contact_popg, ax=ax).get_figure();
group_contact_popg_fig.savefig("POPG Count Temp and Substrate.png", dpi = 300)

In [ ]:
fig_dims = (40, 10)
fig, ax = plt.subplots(figsize=fig_dims)
group_contact_pope_popg_fig = sns.barplot(x = "Joint_Position", y = "POPE_POPG", hue = "Substrate?_Chain", data = group_contact_pope_popg, palette=palette_dic,hue_order=["Yes_A","No_A","Yes_B","No_B"],ax=ax).get_figure()
plt.xlabel("ResID")
plt.ylabel("POPE + POPG Fraction Frames")
plt.legend(loc='upper left');
group_contact_pope_popg_fig.savefig("POPE and POPG Count Sub and Chain.png", dpi = 300)

# Significance Tests for Protein-Lipid Interactions

#### Check if CDL2 Preference Between Substrate vs No Substrate

Keep in mind this result is using all the data! Don't want to do test on data with small points removed - give biased results & skews the test. I made a plot with all the data for substrate & chain to confirm mean plot is correct. Also keep in mind that with the combined temps a bit of power is removed so we're only really detecting the very large differences in sub vs no sub split by chain - this is helpful to only focus on most promising leads. I checked the plots & confirmed they looked reasonable - only very large differences picked up as significant.

In [ ]:
# Extract lipid values based on substrate presence + temp - full data
cdl2_sub_30, cdl2_sub_37, cdl2_nosub_30, cdl2_nosub_37 = extract_lipid_vals_temp_cdl2(group_contact)

In [ ]:
#Extract values from original dataframe - full data
cdl2_sub, cdl2_no_sub = extract_lipid_vals_cdl2(group_contact)

In [ ]:
# Check paritions for sub + temp
len(cdl2_nosub_30)

length cdl2_sub_30 - 209\
length cdl2_sub_37 - 194\
length cdl2_nosub_30 - 212\
length cdl2_nosub_37 - 183


Set these partitions

In [ ]:
# Try bootstrap test for substrate vs no substrate CDL2 at 303.15 K
pooled_cdl2_30_boot = list(np.hstack([cdl2_sub_30,cdl2_nosub_30]))
n_samples = 42
iterations = 1000000
boot_test_cdl2_30 = bootstrap_lipid(pooled_cdl2_30_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram sub vs no sub CDL2 preference at 303.15 K
plt.hist(boot_test_cdl2_30,bins=100);
plt.title("Bootstrap Test CDL2 Preference Substrate vs No Substrate at 303.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram CDL2 Preference Substrate 30 Deg", dpi=150)

In [ ]:
# Try bootstrap test for substrate vs no substrate CDL2 at 310.15 K
pooled_cdl2_37_boot = list(np.hstack([cdl2_sub_37,cdl2_nosub_37]))
n_samples = 38
iterations = 1000000
boot_test_cdl2_37 = bootstrap_lipid(pooled_cdl2_37_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram sub vs no sub CDL2 preference at 310.15 K
plt.hist(boot_test_cdl2_37,bins=100);
plt.title("Bootstrap Test CDL2 Preference Substrate vs No Substrate at 310.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram CDL2 Preference Substrate 37 Deg", dpi=150)

Identical distributions for CDl2 at 303.15 and 310.15 K - can use same distribution and combine temps!

In [ ]:
# Check paritions for sub
len(cdl2_no_sub)

403 values for CDL2 Substrate\
395 values for CDL2 No Substrate

Set these partitions

In [ ]:
# Try bootstrap test for substrate vs no substrate CDL2
pooled_cdl2_boot = list(np.hstack([cdl2_sub,cdl2_no_sub]))
n_samples = 80
iterations = 1000000
boot_test_cdl2 = bootstrap_lipid(pooled_cdl2_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram
plt.hist(boot_test_cdl2,bins=100);
plt.title("Bootstrap Test CDL2 Preference Substrate vs No Substrate")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Boostrap Test Histogram CDL2 Preference Substrate", dpi=150)

In [ ]:
# Separate out substrate and no substrate for cdl2
dic_cdl2_sub, dic_cdl2_no_sub = lipid_dic_cdl2(melt_group_contact_cdl2)

In [ ]:
# Calculate differences in cdl2 values based on protein residue
cdl2_sub_diff, cdl2_sub_unique, cdl2_no_sub_unique = chain_pair_vals(dic_cdl2_sub,dic_cdl2_no_sub)

In [ ]:
#Combine all three dicts to one dictionary
total_cdl2_diff = {**cdl2_sub_diff,**cdl2_sub_unique,**cdl2_no_sub_unique}

In [ ]:
#Actual difference between substrate vs no substrate CDL2 preference
plt.hist(total_cdl2_diff.values(),bins=100);
plt.title("Actual Substrate vs No Substrate CDL2 Preference")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
#plt.savefig("Histogram Substrate vs No Substrate CDL2 Preference", dpi=150)

In [ ]:
#Calculate p-values using bootstrap test distribution for CDL2 preference
cdl2_diff_p_val_boot = p_val(total_cdl2_diff,iterations,boot_test_cdl2)

In [ ]:
# Convert p-values dictionary to dataframe for saving
cdl2_diff_p_val_bootseries = pd.Series(cdl2_diff_p_val_boot, name='p-value')
cdl2_diff_p_val_bootseries.index.name = "Amino Acid"
cdl2_diff_p_val_bootseries.reset_index()

In [ ]:
#save p-values!
cdl2_diff_p_val_bootseries.to_csv('CDL2 Substrate vs No Substrate p-values bootstrap.csv')

In [ ]:
#Filter for significant p-values for CDL2 preference
cdl2_diff_p_val_sig_boot = sig_p_val(cdl2_diff_p_val_boot,0.05)

In [ ]:
cdl2_diff_p_val_sig_boot

11 amino acids with a statistically significant (1%) difference in CDL2 preference between substrate and no substrate based on the bootstrap test:

Chain A:\
N68, T87, K88, T141, R151, R186 (only at 5% sig level), N192, Q268, K369, S402, Q432

Chain B:\
Q432

CDL2 favoured when substrate present (Chain A):\
N68, T87, K88, T141, Q268, K369, S402

CDL2 favoured when substrate not present (Chain A):\
R151, N192, Q432

CDL2 favoured when substrate present (Chain B):\
Q432

CDL2 favoured when substrate not present (Chain B):\
None


#### POPE/POPG Preference Substrate vs No Substrate

In [ ]:
# Extract lipid values based on substrate presence + temp - full data
pope_popg_sub_30, pope_popg_sub_37, pope_popg_nosub_30, pope_popg_nosub_37 = extract_lipid_vals_temp_pope_popg(group_contact)

In [ ]:
#Extract values from original dataframe - full data
pope_popg_sub, pope_popg_no_sub = extract_lipid_vals_pope_popg(group_contact)

In [ ]:
len(pope_popg_nosub_30)

length pope_popg_sub_30 - 209\
length pope_popg_sub_37 - 194\
length pope_popg_nosub_30 - 212\
length pope_popg_nosub_37 - 183

Set these partitions


In [ ]:
# Try bootstrap test for POPE/POPG preference substrate vs no substrate at 303.15 K
pooled_pope_popg_boot = list(np.hstack([pope_popg_sub_30,pope_popg_nosub_30]))
n_samples = 40
iterations = 1000000
boot_test_pope_popg_30 = bootstrap_lipid(pooled_pope_popg_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram at 303.15 K
plt.hist(boot_test_pope_popg_30,bins=100);
plt.title("Bootstrap Test POPE/POPG Preference Substrate vs No Substrate at 303.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram Substrate vs No Substrate POPE POPG Preference 30 Deg", dpi=150)

In [ ]:
# Try bootstrap test for POPE/POPG preference substrate vs no substrate at 310.15 K
pooled_pope_popg_boot_37 = list(np.hstack([pope_popg_sub_37,pope_popg_nosub_37]))
n_samples = 40
iterations = 1000000
boot_test_pope_popg_37 = bootstrap_lipid(pooled_pope_popg_boot_37,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram at 310.15 K
plt.hist(boot_test_pope_popg_37,bins=100);
plt.title("Bootstrap Test POPE/POPG Preference Substrate vs No Substrate at 310.15 K")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram Substrate vs No Substrate POPE POPG Preference 37 Deg", dpi=150)

Identical distributions for CDl2 at 303.15 and 310.15 K - can use same distribution and combine temps!

In [ ]:
len(pope_popg_no_sub)

pope_popg_sub - 403\
pope_popg_no_sub - 395

Set these partitions

In [ ]:
# Get bootstrap test for substrate vs no substrate POPE/POPG preference
pooled_pope_popg_boot = list(np.hstack([pope_popg_sub,pope_popg_no_sub]))
n_samples = 80
iterations = 1000000
boot_test_pope_popg = bootstrap_lipid(pooled_pope_popg_boot,n_samples,iterations)

In [ ]:
#Plot bootstrap test histogram
plt.hist(boot_test_pope_popg,bins=100);
plt.title("Boostrap Test POPE/POPG Preference Substrate vs No Substrate")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Bootstrap Test Histogram POPE and POPG Preference Substrate", dpi=150)

In [ ]:
# Separate out substrate and no substrate for pope/popg
dic_pope_popg_sub, dic_pope_popg_no_sub = lipid_dic_pope_popg(melt_group_contact_pope_popg)

In [ ]:
# Calculate differences in pope/popg values based on protein residue
pope_popg_sub_diff, pope_popg_sub_unique, pope_popg_no_sub_unique = chain_pair_vals(dic_pope_popg_sub,dic_pope_popg_no_sub)

In [ ]:
#Combine all three dicts to one dictionary
total_pope_popg_diff = {**pope_popg_sub_diff,**pope_popg_sub_unique,**pope_popg_no_sub_unique}

In [ ]:
#Actual difference between POPE/POPG preference substrate vs no substrate
plt.hist(total_pope_popg_diff.values(),bins=100);
plt.title("Actual Substrate vs No Substrate POPE/POPG Preference")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
#plt.savefig("Histogram Substrate vs No Substrate POPE POPG Preference", dpi=150)

In [ ]:
#Calculate p-values using bootstrap test distribution for POPE/POPG
pope_popg_diff_p_val_boot = p_val(total_pope_popg_diff,iterations,boot_test_pope_popg)

In [ ]:
# Convert p-values dictionary to dataframe for saving
pope_popg_diff_p_val_series_boot = pd.Series(pope_popg_diff_p_val_boot, name='p-value')
pope_popg_diff_p_val_series_boot.index.name = "Amino Acid"
pope_popg_diff_p_val_series_boot.reset_index()

In [ ]:
#save p-values!
pope_popg_diff_p_val_series_boot.to_csv('Substrate vs No Substrate POPE POPG Preference p-values bootstrap.csv')

In [ ]:
#Filter for significant p-values for bootstrap test
pope_popg_diff_p_val_sig_boot = sig_p_val(pope_popg_diff_p_val_boot,0.05)

In [ ]:
pope_popg_diff_p_val_sig_boot

Set of 21 amino acids with a significant difference (1%) in POPE/POPG preference between substrate vs no substrate for boostrap test:

Chain A:\
Q19, N22, N68, T87, S90, N94, N105, Q111 (5% sig level), S112, T141, S189, T190 (5% sig level), N192, S219 (5% sig level), S272, S312, Q409, Q429, Q435, Q436 

Chain B:\
N192 (5% sig level), S312, S402, Q435

In favour of presence of substrate (Chain A):\
Q19, N22, N68, T87, N94, N105, Q111, S112, T141, N192, S219, S312, Q435, Q436

In favour of presence of substrate (Chain B):\
S312, S402

In favour of no substrate presence (Chain A):\
S90, S189, T190, S272, Q409, Q429

In favour of no substrate presence (Chain B):\
N192, Q435


## Protein-Substrate Contacts

### Import Data

In [ ]:
sub_df_1 = pd.read_csv("data/protein_substrate_contacts_30_deg_traj_1.csv")
sub_df_2 = pd.read_csv("data/protein_substrate_contacts_30_deg_traj_2.csv")
sub_df_3 = pd.read_csv("data/protein_substrate_contacts_37_deg_traj_1.csv")
sub_df_4 = pd.read_csv("data/protein_substrate_contacts_37_deg_traj_2.csv")

In [ ]:
sub_frames = [sub_df_1,sub_df_2,sub_df_3,sub_df_4]
df_substrate = pd.concat(sub_frames, ignore_index=True)

### Extract Amino Acid Type

In [ ]:
df_substrate["Substrate Joint Position"] = df_substrate["Substrate Protein Type"].astype(str) + df_substrate["Substrate Protein ResID"].astype(str)
df_substrate["Acceptor Joint Position"] = df_substrate["Acceptor Protein Type"].astype(str) + df_substrate["Acceptor Protein ResID"].astype(str)

In [ ]:
#sort values by protein position
df_substrate.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True)

In [ ]:
# Calculate mean fraction frames
group_sub_contact = df_substrate.groupby(["ID","Temperature (K)","Acceptor ChainID", "Acceptor Protein ResID","Acceptor Joint Position","Substrate Joint Position"])["Fraction Frames"].mean().unstack()

In [ ]:
#Reset index to remove multi-indexing
group_sub_contact.reset_index(inplace = True)

In [ ]:
#sort values by protein position
group_sub_contact.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True) 

In [ ]:
# Apply melt for later plotting
c_melt = group_sub_contact.melt(id_vars = ["ID","Temperature (K)","Acceptor Protein ResID", "Acceptor Joint Position", "Acceptor ChainID"], value_vars = ["D1","V2","K3","E4","E5","R6","T7",
                                                                                                   "A8","L11","L12","Q13",
                                                                                                   "L14","S15","G16","N17","A18","S19","D20", "F21",
                                                                                                   "S22","Y23"],
                                                                                                   value_name="Fraction_Frames")

In [ ]:
#add temp + chainID column
c_melt["Temperature_AcceptorChainID"] = c_melt["Temperature (K)"].astype(str) + "_" + c_melt["Acceptor ChainID"].astype(str)

##### Label amino acid type by colour - for later plotting

red - negatively charged\
blue - positively charged\
green - hydrophobic\
pink - polar\
yellow - special case (cysteine, glycine, proline)

In [ ]:
amino_colours = {"D1":"red","V2":"green","K3":"blue","E4":"red","E5":"red","R6":"blue","T7":"pink","A8":"green",\
                "A9":"green","S10": "pink","L11": "green","L12":"green","Q13": "pink","L14":"green","S15":"pink",\
                 "G16":"yellow","N17":"pink", "A18":"green","S19":"pink","D20":"red","F21":"green","S22":"pink",\
                 "Y23":"green"}
def apply_amino_colour(string):
    return str(amino_colours[string])

In [ ]:
amino_type = {"D1":"negative charge","V2":"hydrophobic","K3":"positive charge","E4":"negative charge",\
              "E5":"negative charge","R6":"positive charge","T7":"polar","A8":"hydrophobic","A9":"hydrophobic",\
              "S10": "polar","L11": "hydrophobic","L12":"hydrophobic","Q13": "polar","L14":"hydrophobic",\
              "S15":"polar","G16":"polar","N17":"polar", "A18":"hydrophobic","S19":"polar","D20":"negative charge",\
              "F21":"hydrophobic","S22":"polar","Y23":"hydrophobic"}
def apply_amino_type(string):
    return str(amino_type[string])

In [ ]:
c_melt["Substrate Color Column"] = c_melt["Substrate Joint Position"].apply(apply_amino_colour)

In [ ]:
c_melt["Substrate Type Column"] = c_melt["Substrate Joint Position"].apply(apply_amino_type)

In [ ]:
c_melt_nonan = c_melt.dropna(inplace = False) #remove nan values for comparing to lipid interactions/bokeh

In [ ]:
# remove small values for plotting
c_melt_remove = c_melt[c_melt["Fraction_Frames"] > 0.05]
c_melt_nonan_remove = c_melt_nonan[c_melt_nonan["Fraction_Frames"] > 0.05]

In [ ]:
#sort values by protein position
c_melt_remove.sort_values("Acceptor Protein ResID", axis = 0, ascending = True, inplace = True)

### Plotting

In [ ]:
palette_dic_chain = {"303.15_A":blue, "310.15_A":orange,"303.15_B":green,"310.15_B":red}

In [ ]:
chain_C_fig = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                          hue='Temperature_AcceptorChainID', data=c_melt_remove, 
                          palette=palette_dic_chain,
                          hue_order=["303.15_A","310.15_A","303.15_B","310.15_B"],
                          height = 5, aspect = 3).fig;  
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig.savefig("Substrate Interactions by Temp and Chain correct.png", dpi = 300)

In [ ]:
palette_dic = {"negative charge":blue, "positive charge":orange,"polar":green,"hydrophobic":red}

In [ ]:
chain_C_fig_type = sns.catplot(x="Acceptor Joint Position", y="Fraction_Frames", 
                               hue='Substrate Type Column', data=c_melt_remove,
                               palette=palette_dic,
                               hue_order=["hydrophobic","polar","negative charge","positive charge"],
                               height = 5, aspect = 3).fig; 
plt.xlabel('ResID')
plt.ylabel('Fraction Frames')
chain_C_fig_type.savefig("Substrate Interactions by amino acid type correct.png", dpi = 300)

### Bokeh Plot

In [ ]:
from bokeh.plotting import Figure, output_notebook, show, save
from bokeh.models import ColumnDataSource, HoverTool, GroupFilter, CDSView
from bokeh.models import BoxSelectTool

output_notebook()

In [ ]:
c_data = ColumnDataSource(c_melt_nonan_remove)

c_int = Figure(tools='pan,wheel_zoom,box_zoom,reset', tooltips=[('Temperature (K)', '@{Temperature (K)}'), ('Chain', '@{Acceptor ChainID}'),('Sub', '@{Substrate Joint Position}'), ('Acceptor', '@{Acceptor Joint Position}')], width=2000, height=300, x_axis_label = "Acceptor Position", y_axis_label = "Fraction Frames")
c_int.scatter(source=c_data, x='Acceptor Protein ResID', y='Fraction_Frames', \
                  fill_color='Substrate Color Column', size=5)
c_int.add_tools(BoxSelectTool(dimensions="width"))

show(c_int);

##### Save Bokeh Plot

In [ ]:
save(c_int, filename='data/Substrate Interactions Bokeh', resources='inline', title='Substrate Interactions');

### Timeseries of Specific Protein-Substrate Contacts - Thesis

In [ ]:
# read in data - K6 Chain A with D20 Sub
u1_C_term_sub = pd.read_csv("u1 Chain A K6 with Sub D20 Contacts 6 angstrom.csv")
u2_C_term_sub = pd.read_csv("u2 Chain A K6 with Sub D20 Contacts 6 angstrom.csv")
u3_C_term_sub = pd.read_csv("u3 Chain A K6 with Sub D20 Contacts 6 angstrom.csv")
u4_C_term_sub = pd.read_csv("u4 Chain A K6 with Sub D20 Contacts 6 angstrom.csv")

In [ ]:
# read in data - E79 Chain A with R6 Sub
u1_middle_pin_sub = pd.read_csv("u1 Chain A E79 with Sub R6 Contacts 6 angstrom.csv")
u2_middle_pin_sub = pd.read_csv("u2 Chain A E79 with Sub R6 Contacts 6 angstrom.csv")
u3_middle_pin_sub = pd.read_csv("u3 Chain A E79 with Sub R6 Contacts 6 angstrom.csv")
u4_middle_pin_sub = pd.read_csv("u4 Chain A E79 with Sub R6 Contacts 6 angstrom.csv")

In [ ]:
# read in data -  S324 S331 Chain A with Q13 S15 N17 S19 Sub
u1_polar_sub = pd.read_csv("u1 Chain A S324 S331 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")
u2_polar_sub = pd.read_csv("u2 Chain A S324 S331 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")
u3_polar_sub = pd.read_csv("u3 Chain A S324 S331 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")
u4_polar_sub = pd.read_csv("u4 Chain A S324 S331 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")

In [ ]:
# read in data - K51 K58 K61 K62 Chain A with D1 E4 E5 Sub
u1_tail_pin_sub = pd.read_csv("u1 Chain A K51 K58 K61 K62 with Sub D1 E4 E5 Contacts 6 angstrom.csv")
u2_tail_pin_sub = pd.read_csv("u2 Chain A K51 K58 K61 K62 with Sub D1 E4 E5 Contacts 6 angstrom.csv")
u3_tail_pin_sub = pd.read_csv("u3 Chain A K51 K58 K61 K62 with Sub D1 E4 E5 Contacts 6 angstrom.csv")
u4_tail_pin_sub = pd.read_csv("u4 Chain A K51 K58 K61 K62 with Sub D1 E4 E5 Contacts 6 angstrom.csv")

In [ ]:
# read in data - N556 T597 Chain B with Q13 S15 N17 S19 Sub
u1_nbd_sub = pd.read_csv("u1 Chain B N556 T597 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")
u2_nbd_sub = pd.read_csv("u2 Chain B N556 T597 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")
u3_nbd_sub = pd.read_csv("u3 Chain B N556 T597 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")
u4_nbd_sub = pd.read_csv("u4 Chain B N556 T597 with Sub Q13 S15 N17 S19 Contacts 6 angstrom.csv")

In [ ]:
# concatonate by temp - K6 Chain A with D20 Sub
C_term_sub_30_deg = pd.concat([u1_C_term_sub,
                               u2_C_term_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
C_term_sub_37_deg = pd.concat([u3_C_term_sub,
                               u4_C_term_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - E79 Chain A with R6 Sub
middle_pin_sub_30_deg = pd.concat([u1_middle_pin_sub,
                               u2_middle_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
middle_pin_sub_37_deg = pd.concat([u3_middle_pin_sub,
                               u4_middle_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - K51 K58 K61 K62 Chain A with D1 E4 E5 Sub
tail_pin_sub_30_deg = pd.concat([u1_tail_pin_sub,
                               u2_tail_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
tail_pin_sub_37_deg = pd.concat([u3_tail_pin_sub,
                               u4_tail_pin_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - S324 S331 Chain A with Q13 S15 N17 S19 Sub
polar_sub_30_deg = pd.concat([u1_polar_sub,
                               u2_polar_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
polar_sub_37_deg = pd.concat([u3_polar_sub,
                               u4_polar_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# concatonate by temp - N556 T597 Chain B with Q13 S15 N17 S19 Sub
nbd_sub_30_deg = pd.concat([u1_nbd_sub,
                               u2_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))
nbd_sub_37_deg = pd.concat([u3_nbd_sub,
                               u4_nbd_sub]).groupby('Time (ns)').agg(avg=('# Contacts','mean'))

In [ ]:
# Reset index - K6 Chain A with D20 Sub
C_term_sub_30_deg.reset_index(inplace = True)
C_term_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - E79 Chain A with R6 Sub
middle_pin_sub_30_deg.reset_index(inplace = True)
middle_pin_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - K51 K58 K61 K62 Chain A with D1 E4 E5 Sub
tail_pin_sub_30_deg.reset_index(inplace = True)
tail_pin_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - S324 S331 Chain A with Q13 S15 N17 S19 Sub
polar_sub_30_deg.reset_index(inplace = True)
polar_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Reset index - N556 T597 Chain B with Q13 S15 N17 S19 Sub
nbd_sub_30_deg.reset_index(inplace = True)
nbd_sub_37_deg.reset_index(inplace = True)

In [ ]:
# Add in rolling averages - K6 Chain A with D20 Sub
C_term_sub_30_deg_rolling = C_term_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
C_term_sub_37_deg_rolling = C_term_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - E79 Chain A with R6 Sub
middle_pin_sub_30_deg_rolling = middle_pin_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
middle_pin_sub_37_deg_rolling = middle_pin_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - K51 K58 K61 K62 Chain A with D1 E4 E5 Sub
tail_pin_sub_30_deg_rolling = tail_pin_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
tail_pin_sub_37_deg_rolling = tail_pin_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - S324 S331 Chain A with Q13 S15 N17 S19 Sub
polar_sub_30_deg_rolling = polar_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
polar_sub_37_deg_rolling = polar_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Add in rolling averages - N556 T597 Chain B with Q13 S15 N17 S19 Sub
nbd_sub_30_deg_rolling = nbd_sub_30_deg['avg'].rolling(window=1000,center=True).mean()
nbd_sub_37_deg_rolling = nbd_sub_37_deg['avg'].rolling(window=1000,center=True).mean()

In [ ]:
# Plot with rolling averages - K6 Chain A with D20 Sub
#plt.errorbar(x=C_term_sub_30_deg["Time (ns)"],y=C_term_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=C_term_sub_30_deg["Time (ns)"],y=C_term_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=C_term_sub_37_deg["Time (ns)"],y=C_term_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=C_term_sub_37_deg["Time (ns)"],y=C_term_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A K6 with Sub D20")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A K6 with Sub D20 30 Deg", dpi=300)

In [ ]:
# Plot with rolling averages - E79 Chain A with R6 Sub
#plt.errorbar(x=middle_pin_sub_30_deg["Time (ns)"],y=middle_pin_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_sub_30_deg["Time (ns)"],y=middle_pin_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=middle_pin_sub_37_deg["Time (ns)"],y=middle_pin_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=middle_pin_sub_37_deg["Time (ns)"],y=middle_pin_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A E79 with Sub R6")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
#plt.savefig("Average Contacts Chain A E79 with Sub R6", dpi=300)

In [ ]:
# Plot with rolling averages - K51 K58 K61 K62 Chain A with D1 E4 E5 Sub
#plt.errorbar(x=tail_pin_sub_30_deg["Time (ns)"],y=tail_pin_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=tail_pin_sub_30_deg["Time (ns)"],y=tail_pin_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=tail_pin_sub_37_deg["Time (ns)"],y=tail_pin_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=tail_pin_sub_37_deg["Time (ns)"],y=tail_pin_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A K51 K58 K61 K62 with Sub D1 E4 E5")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A K51 K58 K61 K62 with Sub D1 E4 E5", dpi=300)

In [ ]:
# Plot with rolling averages - S324 S331 Chain A with Q13 S15 N17 S19 Sub
#plt.errorbar(x=polar_sub_30_deg["Time (ns)"],y=polar_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=polar_sub_30_deg["Time (ns)"],y=polar_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=polar_sub_37_deg["Time (ns)"],y=polar_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=polar_sub_37_deg["Time (ns)"],y=polar_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain A S324 S331 with Sub Q13 S15 N17 S19")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain A S324 S331 with Sub Q13 S15 N17 S19", dpi=300)

In [ ]:
# Plot with rolling averages - N556 T597 Chain B with Q13 S15 N17 S19 Sub
#plt.errorbar(x=nbd_sub_30_deg["Time (ns)"],y=nbd_sub_30_deg["avg"],label="310 K")
plt.errorbar(x=nbd_sub_30_deg["Time (ns)"],y=nbd_sub_30_deg_rolling,label="303 K")
#plt.errorbar(x=nbd_sub_37_deg["Time (ns)"],y=nbd_sub_37_deg["avg"],label="310 K")
plt.errorbar(x=nbd_sub_37_deg["Time (ns)"],y=nbd_sub_37_deg_rolling,label="310 K")
plt.legend(loc='upper right')
plt.title("Contacts Chain B N556 T597 with Sub Q13 S15 N17 S19")
plt.xlabel("Time (ns)")
plt.ylabel("Average # Contacts")
plt.savefig("Average Contacts Chain B N556 T597 with Sub Q13 S15 N17 S19", dpi=300)

### Membrane Enrichment

Look at using radial distribution curve for each lipid - CDL2, POPE, and POPG and see if it tells us about how these lipids interact in general with target amino acids. Function looks at density of lipids around a target reference.

In [ ]:
# Reimport all contacts for plotting - with substrate
cdl2_sub_30_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_30_Deg_Traj_1.csv")
cdl2_sub_37_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_37_Deg_Traj_1.csv")
cdl2_sub_30_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_30_Deg_Traj_2.csv")
cdl2_sub_37_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Sub_37_Deg_Traj_2.csv")
pope_sub_30_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Sub_30_Deg_Traj_1.csv")
pope_sub_37_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Sub_37_Deg_Traj_1.csv")
pope_sub_30_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Sub_30_Deg_Traj_2.csv")
pope_sub_37_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Sub_37_Deg_Traj_2.csv")
popg_sub_30_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Sub_30_Deg_Traj_1.csv")
popg_sub_37_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Sub_37_Deg_Traj_1.csv")
popg_sub_30_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Sub_30_Deg_Traj_2.csv")
popg_sub_37_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Sub_37_Deg_Traj_2.csv")

In [ ]:
# Reimport all contacts for plotting - without substrate
cdl2_nosub_30_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_30_Deg_Traj_1.csv")
cdl2_nosub_37_deg_traj_1 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_37_Deg_Traj_1.csv")
cdl2_nosub_30_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_30_Deg_Traj_2.csv")
cdl2_nosub_37_deg_traj_2 = pd.read_csv("data/CDL2_Band_Enrichment_Nosub_37_Deg_Traj_2.csv")
pope_nosub_30_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_30_Deg_Traj_1.csv")
pope_nosub_37_deg_traj_1 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_37_Deg_Traj_1.csv")
pope_nosub_30_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_30_Deg_Traj_2.csv")
pope_nosub_37_deg_traj_2 = pd.read_csv("data/POPE_Band_Enrichment_Nosub_37_Deg_Traj_2.csv")
popg_nosub_30_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_30_Deg_Traj_1.csv")
popg_nosub_37_deg_traj_1 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_37_Deg_Traj_1.csv")
popg_nosub_30_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_30_Deg_Traj_2.csv")
popg_nosub_37_deg_traj_2 = pd.read_csv("data/POPG_Band_Enrichment_Nosub_37_Deg_Traj_2.csv")

In [ ]:
# Concat trajectory repeats - substrate
cdl2_sub_30_deg = pd.concat([cdl2_sub_30_deg_traj_1,cdl2_sub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
cdl2_sub_37_deg = pd.concat([cdl2_sub_37_deg_traj_1,cdl2_sub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_sub_30_deg = pd.concat([pope_sub_30_deg_traj_1,pope_sub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_sub_37_deg = pd.concat([pope_sub_37_deg_traj_1,pope_sub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_sub_30_deg = pd.concat([popg_sub_30_deg_traj_1,popg_sub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_sub_37_deg = pd.concat([popg_sub_37_deg_traj_1,popg_sub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))

In [ ]:
# Concat trajectory repeats - no substrate
cdl2_nosub_30_deg = pd.concat([cdl2_nosub_30_deg_traj_1,cdl2_nosub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
cdl2_nosub_37_deg = pd.concat([cdl2_nosub_37_deg_traj_1,cdl2_nosub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_nosub_30_deg = pd.concat([pope_nosub_30_deg_traj_1,pope_nosub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
pope_nosub_37_deg = pd.concat([pope_nosub_37_deg_traj_1,pope_nosub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_nosub_30_deg = pd.concat([popg_nosub_30_deg_traj_1,popg_nosub_30_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))
popg_nosub_37_deg = pd.concat([popg_nosub_37_deg_traj_1,popg_nosub_37_deg_traj_2]).groupby('Band Distance').agg(avg=('Enrichment_Score','mean'), std=('Enrichment_Score','std'))

In [ ]:
#Reset index to remove multi-indexing - substrate
cdl2_sub_30_deg.reset_index(inplace = True)
cdl2_sub_37_deg.reset_index(inplace = True)
pope_sub_30_deg.reset_index(inplace = True) 
pope_sub_37_deg.reset_index(inplace = True) 
popg_sub_30_deg.reset_index(inplace = True) 
popg_sub_37_deg.reset_index(inplace = True) 

In [ ]:
#Reset index to remove multi-indexing - no substrate
cdl2_nosub_30_deg.reset_index(inplace = True)
cdl2_nosub_37_deg.reset_index(inplace = True)
pope_nosub_30_deg.reset_index(inplace = True) 
pope_nosub_37_deg.reset_index(inplace = True) 
popg_nosub_30_deg.reset_index(inplace = True) 
popg_nosub_37_deg.reset_index(inplace = True)

In [ ]:
# Plot by temp for substrate
plt.errorbar(x=cdl2_sub_30_deg["Band Distance"],y=cdl2_sub_30_deg["avg"],yerr=cdl2_sub_30_deg["std"],xerr=2.5,label="CDL2 303.15 K")
plt.errorbar(x=pope_sub_30_deg["Band Distance"],y=pope_sub_30_deg["avg"],yerr=pope_sub_30_deg["std"],xerr=2.5,label="POPE 303.15 K")
plt.errorbar(x=popg_sub_30_deg["Band Distance"],y=popg_sub_30_deg["avg"],yerr=popg_sub_30_deg["std"],xerr=2.5,label="POPG 303.15 K")
plt.errorbar(x=cdl2_sub_37_deg["Band Distance"],y=cdl2_sub_37_deg["avg"],yerr=cdl2_sub_37_deg["std"],xerr=2.5,label="CDL2 310.15 K")
plt.errorbar(x=pope_sub_37_deg["Band Distance"],y=pope_sub_37_deg["avg"],yerr=pope_sub_37_deg["std"],xerr=2.5,label="POPE 310.15 K")
plt.errorbar(x=popg_sub_37_deg["Band Distance"],y=popg_sub_37_deg["avg"],yerr=popg_sub_37_deg["std"],xerr=2.5,label="POPG 310.15 K")
plt.legend(loc='best')
plt.title("Lipid Enrichment with Substrate")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
#plt.savefig("Lipid Enrichment by Temp with Substrate", dpi=300)

In [ ]:
# Plot by temp no substrate
plt.errorbar(x=cdl2_nosub_30_deg["Band Distance"],y=cdl2_nosub_30_deg["avg"],yerr=cdl2_nosub_30_deg["std"],xerr=2.5,label="CDL2 303.15 K")
plt.errorbar(x=pope_nosub_30_deg["Band Distance"],y=pope_nosub_30_deg["avg"],yerr=pope_nosub_30_deg["std"],xerr=2.5,label="POPE 303.15 K")
plt.errorbar(x=popg_nosub_30_deg["Band Distance"],y=popg_nosub_30_deg["avg"],yerr=popg_nosub_30_deg["std"],xerr=2.5,label="POPG 303.15 K")
plt.errorbar(x=cdl2_nosub_37_deg["Band Distance"],y=cdl2_nosub_37_deg["avg"],yerr=cdl2_nosub_37_deg["std"],xerr=2.5,label="CDL2 310.15 K")
plt.errorbar(x=pope_nosub_37_deg["Band Distance"],y=pope_nosub_37_deg["avg"],yerr=pope_nosub_37_deg["std"],xerr=2.5,label="POPE 310.15 K")
plt.errorbar(x=popg_nosub_37_deg["Band Distance"],y=popg_nosub_37_deg["avg"],yerr=popg_nosub_37_deg["std"],xerr=2.5,label="POPG 310.15 K")
plt.legend(loc='best')
plt.title("Lipid Enrichment without Substrate")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment by Temp without Substrate", dpi=300)

In [ ]:
# Plot sub vs no sub 303.15 K
plt.errorbar(x=cdl2_sub_30_deg["Band Distance"],y=cdl2_sub_30_deg["avg"],yerr=cdl2_sub_30_deg["std"],xerr=2.5,label="CDL2 Substrate")
plt.errorbar(x=cdl2_nosub_30_deg["Band Distance"],y=cdl2_nosub_30_deg["avg"],yerr=cdl2_nosub_30_deg["std"],xerr=2.5,label="CDL2 No Substrate")
plt.errorbar(x=pope_sub_30_deg["Band Distance"],y=pope_sub_30_deg["avg"],yerr=pope_sub_30_deg["std"],xerr=2.5,label="POPE Substrate")
plt.errorbar(x=pope_nosub_30_deg["Band Distance"],y=pope_nosub_30_deg["avg"],yerr=pope_nosub_30_deg["std"],xerr=2.5,label="POPE No Substrate")
plt.errorbar(x=popg_sub_30_deg["Band Distance"],y=popg_sub_30_deg["avg"],yerr=popg_sub_30_deg["std"],xerr=2.5,label="POPG Substrate")
plt.errorbar(x=popg_nosub_30_deg["Band Distance"],y=popg_nosub_30_deg["avg"],yerr=popg_nosub_30_deg["std"],xerr=2.5,label="POPG No Substrate")
plt.legend(loc='best')
plt.title("Lipid Enrichment Substrate vs No Substrate at 303.15 K")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment Substrate vs No Substrate 30 Deg", dpi=300)

In [ ]:
# Plot sub vs no sub 310.15 K
plt.errorbar(x=cdl2_sub_37_deg["Band Distance"],y=cdl2_sub_37_deg["avg"],yerr=cdl2_sub_37_deg["std"],xerr=2.5,label="CDL2 Substrate")
plt.errorbar(x=cdl2_nosub_37_deg["Band Distance"],y=cdl2_nosub_37_deg["avg"],yerr=cdl2_nosub_37_deg["std"],xerr=2.5,label="CDL2 No Substrate")
plt.errorbar(x=pope_sub_37_deg["Band Distance"],y=pope_sub_37_deg["avg"],yerr=pope_sub_37_deg["std"],xerr=2.5,label="POPE Substrate")
plt.errorbar(x=pope_nosub_37_deg["Band Distance"],y=pope_nosub_37_deg["avg"],yerr=pope_nosub_37_deg["std"],xerr=2.5,label="POPE No Substrate")
plt.errorbar(x=popg_sub_37_deg["Band Distance"],y=popg_sub_37_deg["avg"],yerr=popg_sub_37_deg["std"],xerr=2.5,label="POPG Substrate")
plt.errorbar(x=popg_nosub_37_deg["Band Distance"],y=popg_nosub_37_deg["avg"],yerr=popg_nosub_37_deg["std"],xerr=2.5,label="POPG No Substrate")
plt.legend(loc=9)
plt.title("Lipid Enrichment Substrate vs No Substrate at 310.15 K")
plt.xlabel("Radial Distance From Protein (Angstrom)")
plt.ylabel("Enrichment")
plt.savefig("Lipid Enrichment Substrate vs No Substrate 37 Deg", dpi=300)

### Lipid Clustering Analysis

Investigate lipid clustering in different domains of HlyB from substrate vs no substrate trajectories

In [ ]:
# Data import with substrate - by domain only
lipid_clust_sub_30_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_30_deg_traj_1.csv")
lipid_clust_sub_30_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_30_deg_traj_2.csv")
lipid_clust_sub_37_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_37_deg_traj_1.csv")
lipid_clust_sub_37_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_sub_37_deg_traj_2.csv")

In [ ]:
# Data import without substrate - by domain only
lipid_clust_nosub_30_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_30_deg_traj_1.csv")
lipid_clust_nosub_30_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_30_deg_traj_2.csv")
lipid_clust_nosub_37_deg_traj_1 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_37_deg_traj_1.csv")
lipid_clust_nosub_37_deg_traj_2 = pd.read_csv("data/Lipid_Clustering_Counts_scaled_nosub_37_deg_traj_2.csv")

In [ ]:
# Data import with substrate - by chain and domain
lipid_clust_sub_30_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_30_deg_traj_1.csv")
lipid_clust_sub_30_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_30_deg_traj_2.csv")
lipid_clust_sub_37_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_37_deg_traj_1.csv")
lipid_clust_sub_37_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_sub_37_deg_traj_2.csv")

In [ ]:
# Data import without substrate - by chain and domain
lipid_clust_nosub_30_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_30_deg_traj_1.csv")
lipid_clust_nosub_30_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_30_deg_traj_2.csv")
lipid_clust_nosub_37_deg_traj_1_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_37_deg_traj_1.csv")
lipid_clust_nosub_37_deg_traj_2_bychain = pd.read_csv("data/Lipid_Clustering_Counts_by_Chain_scaled_nosub_37_deg_traj_2.csv")

In [ ]:
# Combine temps as well & concat
lipid_clust_sub = pd.concat([lipid_clust_sub_30_deg_traj_1,lipid_clust_sub_30_deg_traj_2,lipid_clust_sub_37_deg_traj_1,lipid_clust_sub_37_deg_traj_2],ignore_index=True)
lipid_clust_nosub = pd.concat([lipid_clust_nosub_30_deg_traj_1,lipid_clust_nosub_30_deg_traj_2,lipid_clust_nosub_37_deg_traj_1,lipid_clust_nosub_37_deg_traj_2],ignore_index=True)

In [ ]:
# Combine temps as well & concat - by chain
lipid_clust_sub_bychain = pd.concat([lipid_clust_sub_30_deg_traj_1_bychain,lipid_clust_sub_30_deg_traj_2_bychain,lipid_clust_sub_37_deg_traj_1_bychain,lipid_clust_sub_37_deg_traj_2_bychain],ignore_index=True)
lipid_clust_nosub_bychain = pd.concat([lipid_clust_nosub_30_deg_traj_1_bychain,lipid_clust_nosub_30_deg_traj_2_bychain,lipid_clust_nosub_37_deg_traj_1_bychain,lipid_clust_nosub_37_deg_traj_2_bychain],ignore_index=True)

In [ ]:
# Reset index to remove multi-indexing for combined temps
lipid_clust_sub.reset_index(inplace = True)
lipid_clust_nosub.reset_index(inplace = True)

In [ ]:
# Reset index to remove multi-indexing for combined temps - by domain & chain
lipid_clust_sub_bychain.reset_index(inplace = True)
lipid_clust_nosub_bychain.reset_index(inplace = True)

In [ ]:
# Add in substrate info
lipid_clust_sub["Substrate"] = "yes"
lipid_clust_nosub["Substrate"] = "no"

In [ ]:
# Add in substrate info - by domain & chain
lipid_clust_sub_bychain["Substrate"] = "yes"
lipid_clust_nosub_bychain["Substrate"] = "no"

In [ ]:
# Concat sub and no sub for plotting
lipid_clust = pd.concat([lipid_clust_sub,lipid_clust_nosub])

In [ ]:
# Concat sub and no sub for plotting - by domain & chain
lipid_clust_bychain = pd.concat([lipid_clust_sub_bychain,lipid_clust_nosub_bychain])

In [ ]:
# Add label for seaborn plot
lipid_clust["Label"] = lipid_clust["Substrate"].astype(str) + " " + lipid_clust["Lipid"].astype(str)

In [ ]:
# Add label for seaborn plot - by domain & chain
lipid_clust_bychain["Label"] = lipid_clust_bychain["Substrate"].astype(str) + " " + lipid_clust_bychain["Lipid"].astype(str)

#### Plots

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig_label = sns.barplot(data=lipid_clust, x='Domain', y='Scaled_Median_Count', hue='Label',estimator=np.mean,ci=90).get_figure();
clust_fig_label.savefig("Lipid Clustering", dpi=300)

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig = sns.barplot(data=lipid_clust, x='Domain', y='Scaled_Median_Count', hue='Substrate',estimator=np.mean,ci=90).get_figure();
clust_fig.savefig("Lipid Clustering by substrate", dpi=300)

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig_bychain = sns.barplot(data=lipid_clust_bychain, x='Domain', y='Scaled_Median_Count', hue='Substrate',estimator=np.mean,ci=90).get_figure();
clust_fig_bychain.savefig("Lipid Clustering by substrate and chain", dpi=300)

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
clust_fig_bychain_label = sns.barplot(data=lipid_clust_bychain, x='Domain', y='Scaled_Median_Count', hue='Label',estimator=np.mean,ci=90).get_figure();
clust_fig_bychain_label.savefig("Lipid Clustering by chain", dpi=300)

### Permutation Test - Lipid Clustering

Test permutation test against my data & see differences in the output distributions.

In [ ]:
# Extract values for the test separated by substrate and no substrate
perm_sub_clust, perm_nosub_clust = extract_sub_vals_clust(lipid_clust_bychain)

In [ ]:
#Get permutation test distribution for substrate vs no substrate lipid clustering
pooled = np.hstack([perm_sub_clust,perm_nosub_clust])
sizeA = 96
sizeB= 96
iterations = 1000000
test = permutation(pooled,sizeA,sizeB,iterations)

In [ ]:
#Plot permutation test histogram
plt.hist(test,bins=100);
plt.title("Permutation Test Substrate vs No Substrate")
plt.xlabel("Mean Difference")
plt.ylabel("Frequency");
plt.savefig("Permutation Test Sub vs No Sub Lipid Clustering", dpi=150)

In [ ]:
#Calculate p-values using permutation test distribution
p_val_clust_perm = p_val(diff_lipid_clust,iterations,test)

In [ ]:
#Convert p-values to series for saving
p_val_clust_perm_series = pd.Series(p_val_clust_perm, name='uncorrected p-value')
p_val_clust_perm_series.index.name = "Domain"
p_val_clust_perm_series.reset_index()

In [ ]:
#save p-values!
p_val_clust_perm_series.to_csv('Lipid clustering permutation p-values.csv')

In [ ]:
#Filter for significant p-values for CDL2 preference
perm_clust_p_val_sig = sig_p_val(p_val_clust_perm,0.05)

7 regions which show significant differences (1% sig level) when substrate is present & not present based on the permutation test (p-vals are corrected):

Chain A CLD: CDL2 --> p-val 1.2e-05\
Chain A CLD: POPE --> p-val 0.0\
Chain A CLD: POPG --> p-val 5.4e-05\
Chain A Transmembrane Arm: POPE --> p-val 2.7e-05\
Chain A Transmembrane Top: POPE --> p-val 0.0\
Chain B CLD: POPE --> p-val 0.0\
Chain B Transmembrane Top: POPE --> p-val 1.5e-05

9 regions which show significant differences (5% sig level) when substrate is present & not present based on the permutation test (p-vals are corrected):

Chain A CLD: CDL2 --> p-val 1.2e-05\
Chain A CLD: POPE --> p-val 0.0\
Chain A CLD: POPG --> p-val 5.4e-05\
Chain A Substrate Entrance Channel: POPE --> p-val 0.001655\
Chain A Transmembrane Arm: POPE --> p-val 2.7e-05\
Chain A Transmembrane Top: POPE --> p-val 0.0\
Chain B CLD: POPE --> p-val 0.0\
Chain B Transmembrane Top: POPE --> p-val 1.5e-05\
Chain B Transmembrane Top: POPG --> p-val 0.001811

Regions with lipid depletion when sub not present:\
Chain A CLD --> for CDL2, POPE & POPG\
Chain A Transmembrane Arm --> POPE\
Chain A Transmembrane Top --> POPE\
Chain B CLD --> POPE\
Chain B Transmembrane ToP --> POPE\
Chain B Transmembrane Top --> POPG

Regions with lipid depletion when sub present:\
Chain A Substrate Entrance Channel --> POPE

## SASA Analysis

In [ ]:
# Chain A 30 degrees
chain_A_traj_1_sub_30_deg_x,chain_A_traj_1_sub_30_deg_y = np.loadtxt("data/Traj_1_Sub_30_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_sub_30_deg_x,chain_A_traj_2_sub_30_deg_y = np.loadtxt("data/Traj_2_Sub_30_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_1_nosub_30_deg_x,chain_A_traj_1_nosub_30_deg_y = np.loadtxt("data/Traj_1_NoSub_30_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_nosub_30_deg_x,chain_A_traj_2_nosub_30_deg_y = np.loadtxt("data/Traj_2_NoSub_30_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain B 30 degrees
chain_B_traj_1_sub_30_deg_x,chain_B_traj_1_sub_30_deg_y = np.loadtxt("data/Traj_1_Sub_30_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_sub_30_deg_x,chain_B_traj_2_sub_30_deg_y = np.loadtxt("data/Traj_2_Sub_30_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_1_nosub_30_deg_x,chain_B_traj_1_nosub_30_deg_y = np.loadtxt("data/Traj_1_NoSub_30_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_nosub_30_deg_x,chain_B_traj_2_nosub_30_deg_y = np.loadtxt("data/Traj_2_NoSub_30_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain A 37 degrees
chain_A_traj_1_sub_37_deg_x,chain_A_traj_1_sub_37_deg_y = np.loadtxt("data/Traj_1_Sub_37_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_sub_37_deg_x,chain_A_traj_2_sub_37_deg_y = np.loadtxt("data/Traj_2_Sub_37_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_1_nosub_37_deg_x,chain_A_traj_1_nosub_37_deg_y = np.loadtxt("data/Traj_1_NoSub_37_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_A_traj_2_nosub_37_deg_x,chain_A_traj_2_nosub_37_deg_y = np.loadtxt("data/Traj_2_NoSub_37_deg_Chain_A_CLD_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain B 37 degrees
chain_B_traj_1_sub_37_deg_x,chain_B_traj_1_sub_37_deg_y = np.loadtxt("data/Traj_1_Sub_37_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_sub_37_deg_x,chain_B_traj_2_sub_37_deg_y = np.loadtxt("data/Traj_2_Sub_37_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_1_nosub_37_deg_x,chain_B_traj_1_nosub_37_deg_y = np.loadtxt("data/Traj_1_NoSub_37_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)
chain_B_traj_2_nosub_37_deg_x,chain_B_traj_2_nosub_37_deg_y = np.loadtxt("data/Traj_2_NoSub_37_deg_Chain_B_CLD_sasa.xvg",comments=["@","#"],unpack=True)

In [ ]:
# Chain A 30 degrees data dicts
d_traj_1_chain_A_sub_30_deg = {"Time(ns)":chain_A_traj_1_sub_30_deg_x,'Area':chain_A_traj_1_sub_30_deg_y}
d_traj_2_chain_A_sub_30_deg = {"Time(ns)":chain_A_traj_1_sub_30_deg_x,'Area':chain_A_traj_1_sub_30_deg_y}
d_traj_1_chain_A_nosub_30_deg = {"Time(ns)":chain_A_traj_1_nosub_30_deg_x,'Area':chain_A_traj_1_nosub_30_deg_y}
d_traj_2_chain_A_nosub_30_deg = {"Time(ns)":chain_A_traj_1_nosub_30_deg_x,'Area':chain_A_traj_1_nosub_30_deg_y}

In [ ]:
# Chain B 30 degrees data dicts
d_traj_1_chain_B_sub_30_deg = {"Time(ns)":chain_B_traj_1_sub_30_deg_x,'Area':chain_B_traj_1_sub_30_deg_y}
d_traj_2_chain_B_sub_30_deg = {"Time(ns)":chain_B_traj_1_sub_30_deg_x,'Area':chain_B_traj_1_sub_30_deg_y}
d_traj_1_chain_B_nosub_30_deg = {"Time(ns)":chain_B_traj_1_nosub_30_deg_x,'Area':chain_B_traj_1_nosub_30_deg_y}
d_traj_2_chain_B_nosub_30_deg = {"Time(ns)":chain_B_traj_1_nosub_30_deg_x,'Area':chain_B_traj_1_nosub_30_deg_y}

In [ ]:
# Chain A 37 degrees data dicts
d_traj_1_chain_A_sub_37_deg = {"Time(ns)":chain_A_traj_1_sub_37_deg_x,'Area':chain_A_traj_1_sub_37_deg_y}
d_traj_2_chain_A_sub_37_deg = {"Time(ns)":chain_A_traj_1_sub_37_deg_x,'Area':chain_A_traj_1_sub_37_deg_y}
d_traj_1_chain_A_nosub_37_deg = {"Time(ns)":chain_A_traj_1_nosub_37_deg_x,'Area':chain_A_traj_1_nosub_37_deg_y}
d_traj_2_chain_A_nosub_37_deg = {"Time(ns)":chain_A_traj_1_nosub_37_deg_x,'Area':chain_A_traj_1_nosub_37_deg_y}

In [ ]:
# Chain B 37 degrees data dicts
d_traj_1_chain_B_sub_37_deg = {"Time(ns)":chain_B_traj_1_sub_37_deg_x,'Area':chain_B_traj_1_sub_37_deg_y}
d_traj_2_chain_B_sub_37_deg = {"Time(ns)":chain_B_traj_1_sub_37_deg_x,'Area':chain_B_traj_1_sub_37_deg_y}
d_traj_1_chain_B_nosub_37_deg = {"Time(ns)":chain_B_traj_1_nosub_37_deg_x,'Area':chain_B_traj_1_nosub_37_deg_y}
d_traj_2_chain_B_nosub_37_deg = {"Time(ns)":chain_B_traj_1_nosub_37_deg_x,'Area':chain_B_traj_1_nosub_37_deg_y}

In [ ]:
# Chain A 30 degrees dataframes from dicts
traj_1_chain_A_sub_30_deg = pd.DataFrame(d_traj_1_chain_A_sub_30_deg)
traj_2_chain_A_sub_30_deg = pd.DataFrame(d_traj_2_chain_A_sub_30_deg)
traj_1_chain_A_nosub_30_deg = pd.DataFrame(d_traj_1_chain_A_nosub_30_deg)
traj_2_chain_A_nosub_30_deg = pd.DataFrame(d_traj_2_chain_A_nosub_30_deg)

In [ ]:
# Chain B 30 degrees dataframes from dicts
traj_1_chain_B_sub_30_deg = pd.DataFrame(d_traj_1_chain_B_sub_30_deg)
traj_2_chain_B_sub_30_deg = pd.DataFrame(d_traj_2_chain_B_sub_30_deg)
traj_1_chain_B_nosub_30_deg = pd.DataFrame(d_traj_1_chain_B_nosub_30_deg)
traj_2_chain_B_nosub_30_deg = pd.DataFrame(d_traj_2_chain_B_nosub_30_deg)

In [ ]:
# Chain A 37 degrees dataframes from dicts
traj_1_chain_A_sub_37_deg = pd.DataFrame(d_traj_1_chain_A_sub_37_deg)
traj_2_chain_A_sub_37_deg = pd.DataFrame(d_traj_2_chain_A_sub_37_deg)
traj_1_chain_A_nosub_37_deg = pd.DataFrame(d_traj_1_chain_A_nosub_37_deg)
traj_2_chain_A_nosub_37_deg = pd.DataFrame(d_traj_2_chain_A_nosub_37_deg)

In [ ]:
# Chain B 37 degrees dataframes from dicts
traj_1_chain_B_sub_37_deg = pd.DataFrame(d_traj_1_chain_B_sub_37_deg)
traj_2_chain_B_sub_37_deg = pd.DataFrame(d_traj_2_chain_B_sub_37_deg)
traj_1_chain_B_nosub_37_deg = pd.DataFrame(d_traj_1_chain_B_nosub_37_deg)
traj_2_chain_B_nosub_37_deg = pd.DataFrame(d_traj_2_chain_B_nosub_37_deg)

In [ ]:
# Chain A 30 degrees aggregate by trajectory
chain_A_sub_30_deg = pd.concat([traj_1_chain_A_sub_30_deg,
                               traj_2_chain_A_sub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_A_nosub_30_deg = pd.concat([traj_1_chain_A_nosub_30_deg,
                               traj_2_chain_A_nosub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Chain B 30 degrees aggregate by trajectory
chain_B_sub_30_deg = pd.concat([traj_1_chain_B_sub_30_deg,
                               traj_2_chain_B_sub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_B_nosub_30_deg = pd.concat([traj_1_chain_B_nosub_30_deg,
                               traj_2_chain_B_nosub_30_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Chain A 37 degrees aggregate by trajectory
chain_A_sub_37_deg = pd.concat([traj_1_chain_A_sub_37_deg,
                               traj_2_chain_A_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_A_nosub_37_deg = pd.concat([traj_1_chain_A_nosub_37_deg,
                               traj_2_chain_A_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Chain B 37 degrees aggregate by trajectory
chain_B_sub_37_deg = pd.concat([traj_1_chain_B_sub_37_deg,
                               traj_2_chain_B_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))
chain_B_nosub_37_deg = pd.concat([traj_1_chain_B_nosub_37_deg,
                               traj_2_chain_B_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'))

In [ ]:
# Aggregate by temp - chain A
chain_A_sub = pd.concat([traj_1_chain_A_sub_30_deg,traj_2_chain_A_sub_30_deg,
                         traj_1_chain_A_sub_37_deg,traj_2_chain_A_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))
chain_A_nosub = pd.concat([traj_1_chain_A_nosub_30_deg,traj_2_chain_A_nosub_30_deg,
                         traj_1_chain_A_nosub_37_deg,traj_2_chain_A_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))

In [ ]:
# Aggregate by temp - chain B
chain_B_sub = pd.concat([traj_1_chain_B_sub_30_deg,traj_2_chain_B_sub_30_deg,
                         traj_1_chain_B_sub_37_deg,traj_2_chain_B_sub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))
chain_B_nosub = pd.concat([traj_1_chain_B_nosub_30_deg,traj_2_chain_B_nosub_30_deg,
                         traj_1_chain_B_nosub_37_deg,traj_2_chain_B_nosub_37_deg]).groupby('Time(ns)').agg(avg=('Area','mean'),std=('Area','std'))

In [ ]:
#Reset index to remove multi-indexing - Chain A 30 degrees
chain_A_sub_30_deg.reset_index(inplace = True)
chain_A_nosub_30_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain B 30 degrees
chain_B_sub_30_deg.reset_index(inplace = True)
chain_B_nosub_30_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain A 37 degrees
chain_A_sub_37_deg.reset_index(inplace = True)
chain_A_nosub_37_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain B 37 degrees
chain_B_sub_37_deg.reset_index(inplace = True)
chain_B_nosub_37_deg.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain A
chain_A_sub.reset_index(inplace = True)
chain_A_nosub.reset_index(inplace = True)

In [ ]:
#Reset index to remove multi-indexing - Chain B
chain_B_sub.reset_index(inplace = True)
chain_B_nosub.reset_index(inplace = True)

In [ ]:
# Calulate rolling average - Chain A 30 degrees
chain_A_sub_30_deg_rolling = chain_A_sub_30_deg["avg"].rolling(window=1000,center=True).mean()
chain_A_nosub_30_deg_rolling = chain_A_nosub_30_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain B 30 degrees
chain_B_sub_30_deg_rolling = chain_B_sub_30_deg["avg"].rolling(window=1000,center=True).mean()
chain_B_nosub_30_deg_rolling = chain_B_nosub_30_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain A 37 degrees
chain_A_sub_37_deg_rolling = chain_A_sub_37_deg["avg"].rolling(window=1000,center=True).mean()
chain_A_nosub_37_deg_rolling = chain_A_nosub_37_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain B 37 degrees
chain_B_sub_37_deg_rolling = chain_B_sub_37_deg["avg"].rolling(window=1000,center=True).mean()
chain_B_nosub_37_deg_rolling = chain_B_nosub_37_deg["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain A
chain_A_sub_rolling = chain_A_sub["avg"].rolling(window=1000,center=True).mean()
chain_A_nosub_rolling = chain_A_nosub["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Calulate rolling average - Chain A
chain_B_sub_rolling = chain_B_sub["avg"].rolling(window=1000,center=True).mean()
chain_B_nosub_rolling = chain_B_nosub["avg"].rolling(window=1000,center=True).mean()

In [ ]:
# Plot results - Chain A 30 degrees
plt.errorbar(x=chain_A_sub_30_deg["Time(ns)"],y=chain_A_sub_30_deg["avg"],label="Substrate 303.15 K")
plt.errorbar(x=chain_A_nosub_30_deg["Time(ns)"],y=chain_A_nosub_30_deg["avg"],label="No Substrate 303.15 K")
plt.errorbar(x=chain_A_sub_30_deg["Time(ns)"],y=chain_A_sub_30_deg_rolling)
plt.errorbar(x=chain_A_nosub_30_deg["Time(ns)"],y=chain_A_nosub_30_deg_rolling)
plt.legend(loc='upper left')
plt.title("Chain A Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain A SASA Sub vs No Sub 303.15 K.png", dpi=300)

In [ ]:
# Plot results - Chain B 30 degrees
plt.errorbar(x=chain_B_sub_30_deg["Time(ns)"],y=chain_B_sub_30_deg["avg"],label="Substrate 303.15 K")
plt.errorbar(x=chain_B_nosub_30_deg["Time(ns)"],y=chain_B_nosub_30_deg["avg"],label="No Substrate 303.15 K")
plt.errorbar(x=chain_B_sub_30_deg["Time(ns)"],y=chain_B_sub_30_deg_rolling)
plt.errorbar(x=chain_B_nosub_30_deg["Time(ns)"],y=chain_B_nosub_30_deg_rolling)
plt.legend(loc='upper left')
plt.title("Chain B Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain B SASA Sub vs No Sub 303.15 K.png", dpi=300)

In [ ]:
# Plot results - Chain A 37 degrees
plt.errorbar(x=chain_A_sub_37_deg["Time(ns)"],y=chain_A_sub_37_deg["avg"],label="Substrate 310.15 K")
plt.errorbar(x=chain_A_nosub_37_deg["Time(ns)"],y=chain_A_nosub_37_deg["avg"],label="No Substrate 310.15 K")
plt.errorbar(x=chain_A_sub_37_deg["Time(ns)"],y=chain_A_sub_37_deg_rolling)
plt.errorbar(x=chain_A_nosub_37_deg["Time(ns)"],y=chain_A_nosub_37_deg_rolling)
plt.legend(loc='upper left')
plt.title("Chain A Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain A SASA Sub vs No Sub 310.15 K.png", dpi=300)

In [ ]:
# Plot results - Chain B 37 degrees
plt.errorbar(x=chain_B_sub_37_deg["Time(ns)"],y=chain_B_sub_37_deg["avg"],label="Substrate 310.15 K")
plt.errorbar(x=chain_B_nosub_37_deg["Time(ns)"],y=chain_B_nosub_37_deg["avg"],label="No Substrate 310.15 K")
plt.errorbar(x=chain_B_sub_37_deg["Time(ns)"],y=chain_B_sub_37_deg_rolling)
plt.errorbar(x=chain_B_nosub_37_deg["Time(ns)"],y=chain_B_nosub_37_deg_rolling)
plt.legend(loc='upper left')
plt.title("Chain B Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain B SASA Sub vs No Sub 310.15 K.png", dpi=300)

In [ ]:
# Plot results - Chain A
plt.errorbar(x=chain_A_sub["Time(ns)"],y=chain_A_sub["avg"],yerr=chain_A_sub['std'],label="Substrate")
plt.errorbar(x=chain_A_nosub["Time(ns)"],y=chain_A_nosub["avg"],yerr=chain_A_nosub['std'],label="No Substrate")
plt.errorbar(x=chain_A_sub["Time(ns)"],y=chain_A_sub_rolling,label="Substrate Rolling Average")
plt.errorbar(x=chain_A_nosub["Time(ns)"],y=chain_A_nosub_rolling,label="No Substrate Rolling Average")
plt.legend(loc='upper left')
plt.title("Chain A Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain A SASA Sub vs No Sub label.png", dpi=300)

In [ ]:
# Plot results - Chain B
plt.errorbar(x=chain_B_sub["Time(ns)"],y=chain_B_sub["avg"],yerr=chain_B_sub['std'],label="Substrate")
plt.errorbar(x=chain_B_nosub["Time(ns)"],y=chain_B_nosub["avg"],yerr=chain_B_nosub['std'],label="No Substrate")
plt.errorbar(x=chain_B_sub["Time(ns)"],y=chain_B_sub_rolling,label="Substrate Rolling Average")
plt.errorbar(x=chain_B_nosub["Time(ns)"],y=chain_B_nosub_rolling,label="No Substrate Rolling Average")
plt.legend(loc='upper left')
plt.title("Chain B Solvent Accessible Surface Area")
plt.xlabel("Time (ns)")
plt.ylabel("Area ($nm^2$)")
plt.savefig("Chain B SASA Sub vs No Sub label.png", dpi=300)